## Tutorial: how to import the data and visualise them

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import random
from tqdm import tqdm
from hyperparticle.data_handler import ParticleDataset, OneSet

import ot
import graphicle as gcl

import matplotlib.pyplot as plt
from matplotlib.patches import Circle,ConnectionPatch

#helper functions for plots
from hyperparticle.utils.visualisation import plot_geodesic, get_colors, hard_descendants
from hyperparticle.utils.metrics import sqdist, distance_matrix, precision_and_recall, mAP

from jupyterthemes import jtplot
jtplot.style(theme='oceans16')

## The dataset

The dataset is structured as a Pytorch dataset. The two inputs are the path for the data directory and the signal name (this depends on the physics event we are interested in and it is chosen once the data is generated).

The dataset spits each event as dictionary which contains a graphicle object and a hyperbolic coordinates array for each of the algorithm tree (AntiKt, CA and Kt) and for the MC truth.

In [ ]:
dataset = ParticleDataset('../data/jet_tagging/train_sig/')
event = dataset[4]

In [ ]:
event.keys()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
ax = ax.flatten()

keys = [k for k in event.keys()]
titles = ['Anti-Kt tree', 'CA tree', 'Kt tree',]
for k in range(1, 6, 2):
    i = k//2
    graph, hyp = event[keys[k]], event[keys[k+1]]
    graph, hyp = event[keys[k]], event[keys[k+1]]
    
    G = nx.Graph()
    G.add_edges_from(graph.edges)
    nodes = np.array(G.nodes())
    mapping = {nodes[i]: i for i in range(len(nodes))}
    G = nx.relabel_nodes(G, mapping)
    nodes = np.array(G.nodes())
    for edge in G.edges:
        start = np.where(nodes == edge[0])[0][0]
        end = np.where(nodes == edge[1])[0][0]
        x0,y0 = hyp[start]
        x1,y1 = hyp[end]
        #ax.plot([x0,x1],[y0,y1],alpha=0.1,color='black')
        plot_geodesic(hyp[start],hyp[end],ax[i])

    ax[i].scatter(hyp[:,0], hyp[:,1], c='orange', s=150, alpha=0.6)
    
    disk = Circle((0,0),1,color='white',alpha=0.1)#fill=False)
    ax[i].add_patch(disk)
    ax[i].set_xlim(-1.05,1.05)
    ax[i].set_ylim(-1.05,1.05)
    ax[i].set_title(titles[i])

plt.show()
#plt.savefig('example.png')